In [ ]:
!pip install gradio

In [ ]:
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import gradio as gr

# Cek perangkat (GPU jika tersedia)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load processor dan model BLIP
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# Ukuran standar untuk gambar input
STANDARD_SIZE = (384, 384)

# Fungsi captioning
def generate_captions(image):
    image = image.resize(STANDARD_SIZE).convert("RGB")

    # Conditional caption
    text = "a photography of"
    inputs = processor(image, text, return_tensors="pt").to(device)
    out_cond = model.generate(**inputs)
    caption_conditional = processor.decode(out_cond[0], skip_special_tokens=True)

    # Unconditional caption
    inputs = processor(image, return_tensors="pt").to(device)
    out_uncond = model.generate(**inputs)
    caption_unconditional = processor.decode(out_uncond[0], skip_special_tokens=True)

    return caption_conditional, caption_unconditional

# UI dengan desain modern
with gr.Blocks(theme=gr.themes.Soft(), title="BLIP Image Captioning") as demo:
    gr.Markdown(
        """
        <div style="text-align: center; padding: 10px;">
            <h1 style="font-size: 2.5rem;">🖼️📝 Image Captioning with BLIP</h1>
            <p style="font-size: 1.1rem; color: #444;">
                Upload gambar untuk menghasilkan deskripsi otomatis dengan model <b>BLIP</b>.<br>
                Sistem ini memberikan dua jenis caption:
                <ul style="text-align: left; max-width: 600px; margin: auto;">
                    <li><b>Conditional Caption</b>: dengan prompt awal (<code>a photography of</code>)</li>
                    <li><b>Unconditional Caption</b>: tanpa prompt, sepenuhnya otomatis</li>
                </ul>
            </p>
        </div>
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(
                type="pil",
                label="📷 Upload Image",
                image_mode="RGB",
                width=STANDARD_SIZE[0],
                height=STANDARD_SIZE[1]
            )
            submit_btn = gr.Button("🧠 Generate Captions", variant="primary", size="lg")
        with gr.Column(scale=1):
            conditional_caption = gr.Textbox(label="🎯 Conditional Caption")
            unconditional_caption = gr.Textbox(label="✨ Unconditional Caption")

    submit_btn.click(fn=generate_captions, inputs=image_input, outputs=[conditional_caption, unconditional_caption])

# Jalankan
if __name__ == "__main__":
    demo.launch()
